In [1]:
import pandas as pd
data = pd.read_csv("housePrice (1).csv")
data

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33
...,...,...,...,...,...,...,...,...
3474,86,2,True,True,True,Southern Janatabad,3.500000e+09,116666.67
3475,83,2,True,True,True,Niavaran,6.800000e+09,226666.67
3476,75,2,False,False,False,Parand,3.650000e+08,12166.67
3477,105,2,True,True,True,Dorous,5.600000e+09,186666.67


In [2]:
data= data.drop("Price(USD)", axis = 1)

In [3]:
data.info()

data['Area'] = pd.to_numeric(data['Area'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3479 non-null   object 
 1   Room       3479 non-null   int64  
 2   Parking    3479 non-null   bool   
 3   Warehouse  3479 non-null   bool   
 4   Elevator   3479 non-null   bool   
 5   Address    3456 non-null   object 
 6   Price      3479 non-null   float64
dtypes: bool(3), float64(1), int64(1), object(2)
memory usage: 119.0+ KB


In [ ]:
data.isnull().sum()

Area          6
Room          0
Parking       0
Warehouse     0
Elevator      0
Address      23
Price         0
dtype: int64

In [5]:
data = data.dropna()

In [6]:
data.describe()

,Area,Room,Price
count,3450.000000,3450.000000,3.450000e+03
mean,106.917391,2.081159,5.375563e+09
std,69.550976,0.760216,8.125918e+09
min,30.000000,0.000000,3.600000e+06
25%,69.000000,2.000000,1.419250e+09
50%,90.000000,2.000000,2.900000e+09
75%,120.000000,2.000000,6.000000e+09
max,929.000000,5.000000,9.240000e+10


In [7]:
data['Avg_Price_adrs'] = data.groupby('Address')['Price'].transform('mean')
data['Priceـsquareـmeter']=data['Price']/data['Area']


In [8]:
numerical_cols = data[data.select_dtypes(include=['int64', 'float64']).columns]
data.dtypes

Area                  float64
Room                    int64
Parking                  bool
Warehouse                bool
Elevator                 bool
Address                object
Price                 float64
Avg_Price_adrs        float64
Priceـsquareـmeter    float64
dtype: object

In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
for column in ['Parking', 'Warehouse', 'Elevator', 'Address']:
 data[column] = le.fit_transform(data[column])

In [10]:
# تغییر ستون Price
data['Price'] = data['Price'].apply(lambda x: int(x) if float(x).is_integer() else x)
data

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Avg_Price_adrs,Priceـsquareـmeter
0,63.0,1,1,1,1,156,1850000000,2.985754e+09,2.936508e+07
1,60.0,1,1,1,1,156,1850000000,2.985754e+09,3.083333e+07
2,79.0,2,1,1,1,117,550000000,7.778219e+08,6.962025e+06
3,95.0,2,1,1,1,152,902500000,9.009217e+08,9.500000e+06
4,123.0,2,1,1,1,150,7000000000,1.196988e+10,5.691057e+07
...,...,...,...,...,...,...,...,...,...
3474,86.0,2,1,1,1,163,3500000000,3.178452e+09,4.069767e+07
3475,83.0,2,1,1,1,105,6800000000,1.989321e+10,8.192771e+07
3476,75.0,2,0,0,0,115,365000000,5.235906e+08,4.866667e+06
3477,105.0,2,1,1,1,39,5600000000,1.873462e+10,5.333333e+07


In [11]:
scaler=StandardScaler()
x=data[['Priceـsquareـmeter','Area','Avg_Price_adrs','Room']]
y=data[['Price']]
x_scaled=scaler.fit_transform(x)


In [12]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2 ,random_state=42)
print(x_train.shape)
print(x_test.shape)

(2760, 4)
(690, 4)


In [13]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
from sklearn.metrics import mean_squared_error, r2_score

# مدل Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(x_train, y_train)

# ارزیابی روی تست
y_pred_rf = rf_model.predict(x_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest R2: {r2_rf:.2f}")

c:\Users\NB1\Desktop\proj-AI\myenv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest R2: 0.95


In [14]:
from sklearn.ensemble import RandomForestRegressor
import joblib
joblib.dump(rf_model, 'rf_model.pkl')
print("✅ مدل ذخیره شد در فایل rf_model.pkl")

✅ مدل ذخیره شد در فایل rf_model.pkl
